In [53]:
import numpy as np

In [54]:
MAX_HELICOPTER_CAPACITY = 2000
HELICOPTER_SPEED = 200
MAX_TRUCK_CAPACITY = 1000
TRUCK_SPEED = 100
NUM_DISTRIBUTION_CENTERS = 10
NUM_AFFECTED_CITIES = 11
# 10x11 matrix of distances between distribution centers and affected cities
distance_matrix = np.array([
    [100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600],
    [200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700],
    [300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800],
    [400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900],
    [500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000],
    [600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100],
    [700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200],
    [800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300],
    [900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400],
    [1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500]
])

road_blocked = np.array([
    [False, False, False, True, False, False, False, False, False, False, False],
    [False, False, False, False, True, False, False, False, False, False, False],
    [False, False, False, False, False, True, False, False, False, False, False],
    [True, False, False, False, False, False, True, False, False, False, False],
    [False, True, False, False, False, False, False, True, False, False, False],
    [False, False, True, False, False, False, False, False, True, False, False],
    [False, False, False, True, False, False, False, False, False, True, False],
    [False, False, False, False, True, False, False, False, False, False, True],
    [False, False, False, False, False, True, False, False, False, False, False],
    [False, False, False, False, False, False, True, False, False, False, False]
])

demand_matrix = np.array([
    [100, 150, 200, 120, 80, 90, 140, 160, 110, 130],
    [140, 110, 190, 100, 130, 150, 120, 180, 90, 140],
    [120, 140, 160, 110, 100, 120, 140, 130, 150, 100],
    [160, 120, 130, 150, 140, 110, 100, 130, 140, 120],
    [100, 90, 140, 160, 120, 130, 150, 140, 100, 110],
    [110, 140, 130, 120, 100, 150, 160, 90, 110, 140],
    [90, 100, 120, 130, 140, 110, 150, 140, 130, 100],
    [130, 120, 110, 140, 150, 100, 90, 120, 140, 110],
    [140, 100, 130, 120, 110, 140, 150, 100, 120, 130],
    [120, 140, 110, 100, 130, 120, 140, 160, 110, 100],
    [110, 100, 150, 120, 140, 160, 120, 130, 110, 140]
])

In [55]:


import random

def create_population(size, bounds):
    population = []
    for _ in range(size):
        individual = [
            random.randint(0, MAX_HELICOPTER_CAPACITY),
            random.randint(0, MAX_TRUCK_CAPACITY)
        ]
        population.append(individual)
    return population

def fitness_function(individual):
    helicopter_allocation = individual[0]
    truck_allocation = individual[1]
    
    total_cost = 0

    for i in range(NUM_DISTRIBUTION_CENTERS):
        for j in range(NUM_AFFECTED_CITIES):
            if not road_blocked[i][j]:  # road is not blocked
                distance = distance_matrix[i][j]

                allocated_weight_helicopter = min(helicopter_allocation, demand_matrix[j][i])  # Access demand for affected city j and distribution center i
                allocated_weight_truck = min(truck_allocation, demand_matrix[j][i])  # Access demand for affected city j and distribution center i

                total_cost += distance * (allocated_weight_helicopter / HELICOPTER_SPEED) 
                total_cost += distance * (allocated_weight_truck / TRUCK_SPEED) 
    return total_cost


def selection(population, num_parents):
    parents = random.sample(population, num_parents)
    return parents

def crossover(parents, num_offspring):
    offspring = []
    for _ in range(num_offspring):
        parent1, parent2 = random.sample(parents, 2)
        child = [parent1[0], parent2[1]]
        offspring.append(child)
    return offspring

def mutation(offspring, mutation_rate, bounds):
    for child in offspring:
        if random.random() < mutation_rate:
            child[0] = random.randint(bounds[0], bounds[1])
            child[1] = random.randint(bounds[0], bounds[1])

def genetic_algorithm(bounds, population_size, num_generations, num_parents, num_offspring, mutation_rate):
    population = create_population(population_size, bounds)

    for _ in range(num_generations):
        parents = selection(population, num_parents)
        offspring = crossover(parents, num_offspring)
        mutation(offspring, mutation_rate, bounds)

        population = parents + offspring

        fitness_values = [fitness_function(individual) for individual in population]

        best_fitness = min(fitness_values)
        best_individual = population[fitness_values.index(best_fitness)]

    return best_individual, population

# Parameters
bounds = (0, 1000)
population_size = 100
num_generations = 50
num_parents = 50
num_offspring = 50
mutation_rate = 0.1

# Run the genetic algorithm
best_individual, population = genetic_algorithm(bounds, population_size, num_generations, num_parents, num_offspring, mutation_rate)

print("Best Individual:", best_individual)
print("Best Fitness:", fitness_function(best_individual))


Best Individual: [833, 80]
Best Fitness: 108285.0


In [56]:
def fitness_function(x, y):
    helicopter_allocation = x
    truck_allocation = y
    
    total_cost = 0

    for i in range(NUM_DISTRIBUTION_CENTERS):
        for j in range(NUM_AFFECTED_CITIES):
            if not road_blocked[i][j]:  # road is not blocked
                distance = distance_matrix[i][j]

                allocated_weight_helicopter = min(helicopter_allocation, demand_matrix[j][i])  # Access demand for affected city j and distribution center i
                allocated_weight_truck = min(truck_allocation, demand_matrix[j][i])  # Access demand for affected city j and distribution center i

                total_cost += distance * (allocated_weight_helicopter / HELICOPTER_SPEED) 
                total_cost += distance * (allocated_weight_truck / TRUCK_SPEED) 
    return total_cost

In [57]:
def simulated_annealing(function, bounds, n_iterations, initial_temp, cooling_rate):
    # Initialize a solution randomly within bounds
    current_sol = np.random.uniform(low=bounds[0], high=bounds[1], size=2)
    # Find the cost of the initalized cost
    current_cost = function(*current_sol)

    # At the beginning, initialized solution is best solution, and its cost is best cost
    best_sol, best_cost = current_sol, current_cost
    # Store best solutions
    trajectory = [best_sol]

    # Perform simulated annealing
    for i in range(n_iterations):
        # Generate a candidate solution by adding Gaussian perturbation to the current solution
        candidate_sol = current_sol + np.random.randn(2)
        # Stay withing bounds
        candidate_sol = np.clip(candidate_sol, bounds[0], bounds[1])
        # Cost of the candidate solution
        candidate_cost = function(*candidate_sol)

        # Candidate solution becomes current solution if it has a lower cost 
        # or with a certain probability based on temperature
        if candidate_cost < current_cost or np.random.rand() < np.exp((current_cost - candidate_cost) / initial_temp):
            current_sol, current_cost = candidate_sol, candidate_cost

            # Update the best solution if the candidate has a lower cost
            if candidate_cost < best_cost:
                best_sol, best_cost = candidate_sol, candidate_cost
                # Append current best solution to trajectory list
                trajectory.append(best_sol)

        # Cool down the temperature
        initial_temp *= cooling_rate
#     Return best solution, best cost, and array that we store best solutions
    return best_sol, best_cost, trajectory

bounds = (0, 1000)
n_iterations = 1000
initial_temp = 1000
cooling_rate = 0.9

best_allocation, best_fitness, trajectory = simulated_annealing(fitness_function, bounds, n_iterations, initial_temp, cooling_rate)

print("Best Allocation:", best_allocation)
print("Best Fitness:", best_fitness)

Best Allocation: [  0.        930.1418626]
Best Fitness: 95210.0
